# 气象超算本地文件

本节介绍如何使用 reki 内置的配置文件查找 CMA 国家级气象超算子系统 1 (CMA-HPC2023-SC1) 上的 CEMC 业务系统数据文件路径。

首先加载需要使用的模块

In [2]:
import pandas as pd
import numpy as np
import xarray as xr

from reki.data_finder import find_local_file

生成起报时次和预报时效

In [3]:
start_time = pd.Timestamp.utcnow().floor(freq="D") - pd.Timedelta(days=2)
start_time_label = start_time.strftime("%Y%m%d%H")
forecast_time_label = "24h"
forecast_time = pd.to_timedelta(forecast_time_label)
print("start_time:", start_time)
print("start_time_label:", start_time_label)
print("forecast_time:", forecast_time)
print("forecast_time_label:", forecast_time_label)

start_time: 2025-07-01 00:00:00+00:00
start_time_label: 2025070100
forecast_time: 1 days 00:00:00
forecast_time_label: 24h


## 基本用法

`reki.data_finder.find_local_file` 函数根据内置的配置文件查找本地文件。

查找 CMA-GFS GRIB2 数据文件路径，使用如下参数

- data_type: 数据类型，用于定位配置文件路径，这里使用内置的数据类型 `"cma_gfs_gmf/grib2/orig"`
- start_time: 起报时次，格式 YYYYMMDDHH
- forecast_time: 预报时效，通常为小时，理论上也支持分钟级别输出

In [4]:
file_path = find_local_file(
    "cma_gfs_gmf/grib2/orig",
    start_time=start_time,
    forecast_time=forecast_time,
)
file_path

PosixPath('/g3/COMMONDATA/OPER/CEMC/GFS_GMF/Prod-grib/2025070100/ORIG/gmf.gra.2025070100024.grb2')

查找 CMA-GFS 预处理后的观测资料数据，可以使用 `obs_time` 指定观测资料对应的时间。
CMA-GFS 使用起报时刻前后3小时的观测资料。

In [13]:
file_path = find_local_file(
    "cma_gfs_gmf/obs/rsurf",
    start_time=start_time,
    obs_time=start_time - pd.Timedelta(hours=1, minutes=30)
)
file_path

PosixPath('/g3/COMMONDATA/OPER/CEMC/GFS_GMF/Obs-prep/2025070100/rec_RSURF_20250630212230_g.dat')

In [12]:
file_path = find_local_file(
    "cma_gfs_gmf/obs/rsurf",
    start_time=start_time,
    obs_time=start_time + pd.Timedelta(hours=1, minutes=30)
)
file_path

PosixPath('/g3/COMMONDATA/OPER/CEMC/GFS_GMF/Obs-prep/2025070100/rec_RSURF_20250630210130_g.dat')

查找 CMA-GEPS GRIB2 数据文件路径，在上述参数基础上需要使用 `number` 参数指定集合成员编号。
其中 0 表示控制预报，1-31 表示 31 个集合成员。

In [ ]:
file_path = find_local_file(
    "cma_geps/grib2/orig",
    start_time=start_time,
    forecast_time="24h",
    number=0,
)
file_path

PosixPath('/g3/COMMONDATA/OPER/CEMC/GEPS/Prod-grib/2025070100/grib2/gef.gra.000.2025070100024.grb2')

In [ ]:
file_path = find_local_file(
    "cma_geps/grib2/orig",
    start_time=start_time,
    forecast_time="24h",
    number=14,
)
file_path

PosixPath('/g3/COMMONDATA/OPER/CEMC/GEPS/Prod-grib/2025070100/grib2/gef.gra.014.2025070100024.grb2')

`find_local_file` 默认检索数值预报业务系统数据，即 `data_class="od"`。
如果检索其他类型数据，需要手动设置 `data_class` 参数。

比如检索 NCEP GFS 数据，需要设置 `data_class="cm"`。

In [ ]:
file_path = find_local_file(
    "glob/gfs/grib2/0p50",
    data_class="cm",
    start_time=start_time,
    forecast_time=forecast_time,
)
file_path

file name: gfs.t00z.pgrb2.0p50.f024
searching file path: /g3/COMMONDATA/glob/gfs/2025/gfs.2025070100/gfs.t00z.pgrb2.0p50.f024


PosixPath('/g3/COMMONDATA/glob/gfs/2025/gfs.2025070100/gfs.t00z.pgrb2.0p50.f024')

如果想要查看 `find_local_file` 函数的具体查找文件过程，可以使用 `debug=True` 参数。

In [17]:
file_path = find_local_file(
    "glob/gfs/grib2/1p00",
    data_class="cm",
    start_time=start_time,
    forecast_time=forecast_time,
    debug=True,
)
file_path

file name: gfs.t00z.pgrb2.1p00.f024
searching file path: /g3/COMMONDATA/glob/gfs/2025/gfs.2025070100/gfs.t00z.pgrb2.1p00.f024


PosixPath('/g3/COMMONDATA/glob/gfs/2025/gfs.2025070100/gfs.t00z.pgrb2.1p00.f024')

## 更多函数

`reki.data_finder.get_local_file_name()` 函数返回文件名，但不实际查找文件是否存在。

In [19]:
from reki.data_finder import get_local_file_name

file_name = get_local_file_name(
    "cma_meso_1km/grib2/orig",
    start_time=start_time,
    forecast_time=forecast_time,
)
file_name

'rmf.hgra.2025070100024.grb2'